# Integration tets

In [115]:
import requests
from data_generators.generators import RandomMentor

In [116]:
url = "http://127.0.0.1:8000"

## Mentor Operations

In [117]:
# URL for the request
url_create_mentor = url + '/create/mentor'
url_delete_mentor = url + '/delete/mentor'
url_read_mentor = url + '/read/mentor'

### Create Mentor
POST `/create/mentor`

### Setup for Mentors
Create a list of randomly generated mock mentor data and mark them as 'test_INTEGRATION_test' for post-testing cleanup.

In [118]:
# Set number of mentors to test
n_mentors = 3
# Create list of test mentors
mentors = []

for i in range(n_mentors):
    mentors.append(vars(RandomMentor()))
    mentors[i]['other_info'] = 'test_INTEGRATION_test'


Use `/create/mentors/` endpoint to create a single mentor using properly formed mock data.
Verify that the status code returns 200 [successful], and that the expected JSON values are returned.

In [119]:
# Create a ONE test mentor with API, we should receive 'result': True
expected = {"result": True}
ret = requests.post(url_create_mentor, json=mentors[0])

assert ret.status_code == 200, f"Return status code was not 200. Got status code {ret.status_code}"
assert expected == ret.json(), "Endpoint did not return expected value of {\"result\": True}"

2. Failed test. Create multiple test mentors with API
API Should return *error*

In [79]:
# API doesn't allow to create multiples entries
requests.post(url_create_mentor, json=mentors).json()


{'detail': [{'loc': ['body'],
   'msg': 'value is not a valid dict',
   'type': 'type_error.dict'}]}

4. Failed test. Try to create duplicated recrord  
API Should return:  
`{'detail': 'Profile ID must be unique.'}`

In [80]:
# Try to create a DUPLICATE test mentor with API, we should receive 'result': True
requests.post(url_create_mentor, json=mentors[0]).json()


{'detail': 'Profile ID must be unique.'}

5. Failed test. Try to create record with field not in schema.  
API should return *error*  `value_error.extra`

In [81]:
# Create an extra test field
mentors[1]['test_FIELED_test'] = 'This a field for integration test'

# Try to create mentor
requests.post(url_create_mentor, json=mentors[1]).json()


{'detail': [{'loc': ['body', 'test_FIELED_test'],
   'msg': 'extra fields not permitted',
   'type': 'value_error.extra'}]}

6. Failed test. Try to create a record with wrong value  
API should return 422 error:  
 `value_error.any_str.max_length`

In [82]:
# Create First name more than 255 legal chars
mentors[2]['first_name'] = 300 * 'a'

# Try to create a wrong mentor
requests.post(url_create_mentor, json=mentors[2]).json()


{'detail': [{'loc': ['body', 'first_name'],
   'msg': 'ensure this value has at most 255 characters',
   'type': 'value_error.any_str.max_length',
   'ctx': {'limit_value': 255}}]}

### Read Mentor
POST `/read/mentor`

In [83]:
url_read_mentor = url + '/read/mentor'


1. Read one test mentor by querying `profile_id`

In [84]:
# Read first test mentror with unique profile_id
requests.post(url_read_mentor, json={'profile_id': mentors[0]['profile_id']}).json()

{'result': [{'profile_id': 'fdb30e87-c0b7-4f83-99e7-8c2f99f236ed',
   'first_name': 'Lochlan',
   'last_name': 'Murphy',
   'email': 'Lochlan.Murphy@gmail.com',
   'country': 'U.S.',
   'state': 'Georgia',
   'city': 'Rochester',
   'current_company': 'Amazon',
   'current_position': 'Data Scientist',
   'tech_stack': ['Data Science', 'Career Development'],
   'job_help': True,
   'industry_knowledge': True,
   'pair_programming': True,
   'commitment': True,
   'referred_by': 'Family',
   'other_info': 'test_INTEGRATION_test',
   'validate_status': 'pending',
   'is_active': True,
   'accepting_new_mentees': True,
   'created_at': '2022-10-08T20:03:05.204000'}]}

2. Read all entrees  
We should get the same data as we get through DB request

In [85]:
# read from API, with no parameteres, we should get all the data
df_api = DataFrame(requests.post(url_read_mentor).json()['result'])

# read from DB
df_db = DataFrame(db.read('Mentors'))

# If there are some difference in data we should get a dataframe with a number of rows of different entrees
assert len(df_api.compare(df_db)) == 0

3. Failed test. Read inexisting value.  
Should return zero list

In [86]:
# Read from the API with an inexisting name
requests.post(url_read_mentor, json={'first_name': 'test_INEXISTINGNAME_test'}).json()

{'result': []}

### Update Mentors
POST `/update/mentor/{profile_id}`

In [87]:
url_update_mentor = f'{url}/update/mentor/'


1. Update existing test Mentor

In [88]:
# Update test Mentor current_company value
requests.post(url_update_mentor  + mentors[0]['profile_id'], json={'current_company': 'test_Bloomtech_test'}).json()

# Read the query with our test value
requests.post(url_read_mentor, json={'current_company': 'test_Bloomtech_test'}).json()

{'result': [{'profile_id': 'fdb30e87-c0b7-4f83-99e7-8c2f99f236ed',
   'first_name': 'Lochlan',
   'last_name': 'Murphy',
   'email': 'Lochlan.Murphy@gmail.com',
   'country': 'U.S.',
   'state': 'Georgia',
   'city': 'Rochester',
   'current_company': 'test_Bloomtech_test',
   'current_position': 'Data Scientist',
   'tech_stack': ['Data Science', 'Career Development'],
   'job_help': True,
   'industry_knowledge': True,
   'pair_programming': True,
   'commitment': True,
   'referred_by': 'Family',
   'other_info': 'test_INTEGRATION_test',
   'validate_status': 'pending',
   'is_active': True,
   'accepting_new_mentees': True,
   'created_at': '2022-10-08T20:03:05.204000',
   'updated_at': '2022-10-08T20:03:07.067000'}]}

2. Failed test. Try to update with wrong value

In [89]:
# Create a wrong value (more than 255 chars)
current_position_wrong = 300 * 'a'

# Update test Mentor current_company value
# Shoul return value_errror
requests.post(url_update_mentor  + mentors[0]['profile_id'], json={'current_company': current_position_wrong}).json()



{'detail': [{'loc': ['body', 'current_company'],
   'msg': 'ensure this value has at most 255 characters',
   'type': 'value_error.any_str.max_length',
   'ctx': {'limit_value': 255}}]}

#### Clean the data

In [90]:
# Remove all test mentors from db
[db.delete('Mentors', {'profile_id': mentor['profile_id']}) for mentor in mentors]

[None, None, None]

In [91]:
df_mentors.compare(DataFrame(db.read('Mentors')))

Empty DataFrame
Columns: []
Index: []

In [92]:
# Compare original df with a new one
# I was trying to compare 
# assert len(df_mentors.compare(DataFrame(db.read('Mentors')))) == 0
assert df_mentors.shape == DataFrame(db.read('Mentors')).shape

In [93]:
# query all test_*_test entrees to check that we delete everythinb
...

## Mentee Operations



In [94]:
df_mentees = DataFrame(db.read('Mentees'))

### Create Mentee
POST `/create/mentee`

In [95]:
url_create_mentee = f'{url}/create/mentee'

1. Generate a list of random *test* mentees

In [96]:
# Set number of mentors to test
n_mentees = 3
# Create list of test mentors
mentees = []

for i in range(n_mentees): 
    mentees.append(vars(RandomMentee()))

    # create a field to identify our test records later
    mentees[i]['other_info'] = 'test_INTEGRATION_test'


2. Create *one* test mentee with API

In [97]:
requests.post(url_create_mentee, json=mentees[0]).json()


{'result': True}

3. Failed test. Create multiple test mentees with API  
API Should return *error*

In [98]:
# API doesn't allow to create multiples entries
requests.post(url_create_mentee, json=mentees).json()


{'detail': [{'loc': ['body'],
   'msg': 'value is not a valid dict',
   'type': 'type_error.dict'}]}

4. Failed test. Try to create duplicated recrord  
API Should return:  
`{'detail': 'Profile ID must be unique.'}`

In [99]:
# Try to create a DUPLICATE test mentor with API, we should receive 'result': True
requests.post(url_create_mentee, json=mentees[0]).json()


{'detail': 'Profile ID must be unique.'}

5. Failed test. Try to create record with field not in schema.  
API should return *error*  `value_error.extra`

In [100]:
# Create an extra test field
mentees[1]['test_FIELED_test'] = 'This a field for integration test'

# Try to create mentor
requests.post(url_create_mentee, json=mentees[1]).json()


{'detail': [{'loc': ['body', 'test_FIELED_test'],
   'msg': 'extra fields not permitted',
   'type': 'value_error.extra'}]}

6. Failed test. Try to create a record with wrong value  
API should return 422 error:  
 `value_error.any_str.max_length`

In [101]:
# Create First name more than 255 legal chars
mentees[2]['first_name'] = 300 * 'a'

# Try to create a wrong mentor
requests.post(url_create_mentee, json=mentees[2]).json()


{'detail': [{'loc': ['body', 'first_name'],
   'msg': 'ensure this value has at most 255 characters',
   'type': 'value_error.any_str.max_length',
   'ctx': {'limit_value': 255}}]}

### Read mentee
POST `/read/mentee`

In [102]:
url_read_mentee = f'{url}/read/mentee'

1. Read one test mentee by querying `profile_id`

In [103]:
# Read first test mentror with unique profile_id
requests.post(url_read_mentee, json={'profile_id': mentees[0]['profile_id']}).json()

{'result': [{'profile_id': '49fdc00a-9aaf-4c0a-969b-e448de1656be',
   'first_name': 'Tomas',
   'last_name': 'Kim',
   'email': 'Tomas.Kim@gmail.com',
   'country': 'U.S.',
   'state': 'Guam',
   'city': 'Springfield',
   'formerly_incarcerated': False,
   'underrepresented_group': False,
   'low_income': True,
   'convictions': 'Misdemeanor, Felony',
   'tech_stack': 'iOS',
   'job_help': False,
   'pair_programming': False,
   'referred_by': 'YouTube',
   'other_info': 'test_INTEGRATION_test',
   'validate_status': 'pending',
   'is_active': False,
   'in_project_underdog': False,
   'created_at': '2022-10-08T20:03:09.389000'}]}

2. Read all entrees  
We should get the same data as we get through DB request

In [104]:
# read from API
df_api = DataFrame(requests.post(url_read_mentee).json()['result'])

# read from DB
df_db = DataFrame(db.read('Mentees'))

# test
assert len(df_api.compare(df_db)) == 0

3. Failed test. Read inexisting value.  
Should return zero list

In [105]:
# Read from the API with an inexisting name
requests.post(url_read_mentee, json={'first_name': 'test_INEXISTINGNAME_test'}).json()

{'result': []}

### Update Mentees
POST `/update/mentee/{profile_id}`

In [106]:
url_update_mentee = f'{url}/update/mentee/'


1. Update existing test Mentor

In [107]:
# Update test Mentor current_company value
requests.post(url_update_mentee  + mentees[0]['profile_id'], json={'tech_stack': 'test_Bloomtech_test'}).json()

# Read the query with our test value
requests.post(url_read_mentee, json={'tech_stack': 'test_Bloomtech_test'}).json()

{'result': [{'profile_id': 'fb5c1f81-2249-427d-a738-7f60cbd32ecb',
   'first_name': 'Lian',
   'last_name': 'Hill',
   'email': 'Lian.Hill@gmail.com',
   'country': 'U.S.',
   'state': 'American Samoa',
   'city': 'Arrowhead',
   'formerly_incarcerated': True,
   'underrepresented_group': False,
   'low_income': False,
   'convictions': 'Felony, Misdemeanor',
   'tech_stack': 'test_Bloomtech_test',
   'job_help': False,
   'pair_programming': False,
   'referred_by': 'Family',
   'other_info': 'test_INTEGRATION_test',
   'validate_status': 'approved',
   'is_active': True,
   'in_project_underdog': False,
   'created_at': '2022-10-07T14:51:55.785000',
   'updated_at': '2022-10-07T15:05:51.655000'},
  {'profile_id': 'ec6ee51c-4a5d-4794-a534-c92f4f6121ad',
   'first_name': 'Marvin',
   'last_name': 'Carter',
   'email': 'Marvin.Carter@gmail.com',
   'country': 'U.S.',
   'state': 'Iowa',
   'city': 'Springfield',
   'formerly_incarcerated': True,
   'underrepresented_group': True,
   'lo

## Feedback Operations

### Create Feedback
POST `/create/feedback`

### Read Feedback
POST `/read/feedback`

### Update Feedback
PATCH `/update/feedback`

### Delete feedback
DELETE `/delete/feedback`

## Graph Operations

### Tech Stack By Role
GET `/graph/tech-stack-by-role`

## Model Operations
POST `/match{profile_id}`